# Conjugate gratients

Here, we implement an iterative numerical solver, called *conjugate gradient algorithm*, to solve a system of linear equations

\begin{equation}
Ax = b,
\end{equation}

where $A \in \mathbb{R}^{n \times n}$ is a symmetric positive-definite matrix and $x$ and $b$ are real vectors of suitable sizes.

In [1]:
import numpy as np
import scipy as sp
from scipy import stats
from scipy import linalg

First let's generate some data. Since $A$ needs to be postive definite (pd), we just sample a Wishart random variable which is guaranteed to be at least positive semi-definite (psd). If we then compute its square we get a pd matrix.

In [2]:
N = 5
np.random.seed(23)

A = stats.wishart.rvs(1, .5, size=(N, N), random_state=None)
A = sp.dot(A.T, A)
A

array([[0.83278163, 1.26179175, 0.51755842, 1.0452182 , 0.67888043],
       [1.26179175, 3.84984806, 2.03011661, 2.12923287, 2.59537078],
       [0.51755842, 2.03011661, 4.1467484 , 0.63641981, 2.26276269],
       [1.0452182 , 2.12923287, 0.63641981, 1.53401186, 1.38406953],
       [0.67888043, 2.59537078, 2.26276269, 1.38406953, 4.14337156]])

We can check if a matrix is pd by computing its eigen values. If all of them are greater zero the matrix is positive definite.

In [3]:
print("Matrix is pd: ", sp.all(sp.linalg.eigvals(A) > 0))

Matrix is pd:  True


Next let's generate a random vector $x$ which we try to estimate later. We use this in order to compute $b$.

In [4]:
x = sp.random.randn(N)
x

array([-0.07132392, -0.04543758,  1.04088597, -0.09403473, -0.42084395])

In [5]:
b = A.dot(x)
b

array([ 0.03799975,  0.55572826,  3.1750188 , -0.23558348,  0.31506669])

The next part is the implentation of the conjugate gradient algorithm. It's is rather short

In [6]:
xk = sp.random.randn(N)
rk = A.dot(xk) - b
pk = -rk

while any(rk > 0.000001):
    alpha = rk.T.dot(rk) / pk.T.dot(A).dot(pk)
    xk = xk + alpha * pk
    rd = rk.T.dot(rk)
    rk = rk + alpha * A.dot(pk)
    beta = rk.T.dot(rk) / rd
    pk = -rk + beta * pk

Let's check if our estimated $x$ vector is the same as the original $x$. Since we expect the solution to be unique, the two vectors should be almost identical.

In [7]:
print(x)
print(xk)

[-0.07132392 -0.04543758  1.04088597 -0.09403473 -0.42084395]
[-0.07132392 -0.04543758  1.04088597 -0.09403473 -0.42084395]


In [8]:
print(A.dot(x))
print(A.dot(xk))

[ 0.03799975  0.55572826  3.1750188  -0.23558348  0.31506669]
[ 0.03799975  0.55572826  3.1750188  -0.23558348  0.31506669]


Cool, the two solutions are absolutely identical.